In [ ]:
#1)

#This script resets the AWA records when the weightclasses or records get reset
#The output is the records sheets with the standards in the correct form for the AWA website
#This notebook is meant to be ran in a Google Drive/Google Colab enviornment

#All modifications needed can be done in cell 3

#Last modified on November 29th, 2025 by M. H. Kent

In [ ]:
#2)

#Import needed libraries to run the script

#Install needed packages that are not already built into Google Drive
!pip install XlsxWriter

#Import needed packages
import pandas as pd
import numpy as np
import glob
import os
import math
from datetime import datetime
from google.colab import drive
from google.colab import files
from google.colab import auth
import openpyxl as opxl
import xlsxwriter


In [ ]:
#3)

#Set needed parameters and paths to build and export a new sheet

#Specify if we want to export the sheet
#Set to "True" if we want to export the sheet, set to "False" otherwise
to_exp = True
#If true, update the path in the line below

#Sets a path to the records sheets
AWA_new_rec_dump = "Your drive path here"

#The Canadian nationals standards. These are the same as the INT2 standards
yth_M_st = [192, 202, 214, 226, 242, 256, 264, 282]
yth_F_st = [119, 128, 138, 147, 155, 164, 173, 181]
jr_M_st = [223, 236, 249, 266, 282, 290, 310, 342]
jr_F_st = [141, 152, 162, 171, 180, 191, 200, 224]
sr_M_st = [240, 254, 269, 287, 304, 313, 335, 369]
sr_F_st = [153, 164, 175, 185, 194, 206, 216, 242]
#Note that the qualifing standards can be found here https://weightliftingcanada.ca/resources/qualifying-standards-for-international-events/

#Define the percent of the INT2 standard that we are going off of. Ranges from 0-100
sr_perc = 80
jr_perc = 80
yth_perc = 80
mast_perc = 80

#Define jr sr and masters weightclasses
weight_classes_male = ["60kg", "65kg", "71kg", "79kg", "88kg", "94kg", "110kg", "+110kg"]
weight_classes_female = ["48kg", "53kg", "58kg", "63kg", "69kg", "77kg", "86kg", "+86kg"]

#Define youth weightclasses
yth_weight_classes_male = ["56kg" ,"60kg", "65kg", "71kg", "79kg", "88kg", "94kg", "+94kg"]
yth_weight_classes_female = ["44kg","48kg", "53kg", "58kg", "63kg", "69kg", "77kg", "+77kg"]

#Snatch to clean and jerk ratio
sn_cj_rat = .8

#Define sexes
sex = ["M", "W"]

#Define masters age categories
mst_ages = [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]

#Define the column names of the records sheets
AB_format_cnames = ["Category", "Weight Cat.", "Lift", "Last Name", "First Name", "Club", "Record", "Event", "Location", "Date"]

#Define lifts
liftsAWA = ["Snatch", "Clean&Jerk", "Total"]

#Defining the masters age factors for Q points
agefac_dat = pd.DataFrame({
    "Age": list(range(30, 91)),
    "AgeFactor_Women": [
        1.000, 1.010, 1.021, 1.031, 1.042, 1.052, 1.063, 1.073, 1.084, 1.096,
        1.108, 1.122, 1.138, 1.155, 1.173, 1.194, 1.216, 1.240, 1.265, 1.292,
        1.321, 1.352, 1.384, 1.419, 1.456, 1.494, 1.534, 1.575, 1.617, 1.660,
        1.704, 1.748, 1.794, 1.841, 1.890, 1.942, 1.996, 2.052, 2.109, 2.168,
        2.226, 2.285, 2.343, 2.402, 2.464, 2.528, 2.597, 2.670, 2.749, 2.831,
        2.918, 3.009, 3.104, 3.201, 3.301, 3.403, 3.507, 3.613, 3.720, 3.827,
        3.935
    ],
    "AgeFactor_Men": [
        1.000, 1.010, 1.018, 1.026, 1.038, 1.052, 1.064, 1.076, 1.088, 1.100,
        1.112, 1.124, 1.136, 1.148, 1.160, 1.173, 1.187, 1.201, 1.215, 1.230,
        1.247, 1.264, 1.283, 1.304, 1.327, 1.351, 1.376, 1.401, 1.425, 1.451,
        1.477, 1.504, 1.531, 1.560, 1.589, 1.620, 1.654, 1.693, 1.736, 1.784,
        1.833, 1.883, 1.932, 1.981, 2.031, 2.083, 2.139, 2.202, 2.271, 2.348,
        2.430, 2.524, 2.635, 2.755, 2.877, 3.008, 3.168, 3.356, 3.545, 3.709,
        3.880
    ]
})
#print(agefac_dat)
#Masters agefactor data was found here: https://osf.io/u56fm


In [ ]:
#4)

#Conect to the drive
#Unhash if not mounted
#Note that if we do not plan to export, we do not mount because everything needed to build the sheet is in the code
if to_exp == True:
  #Authenticate to the user
  auth.authenticate_user()
  #Mount to the drive
  drive.mount('/content/drive')

In [ ]:
#5)

#Define needed internal functions for building the sheets

def q_points_male(total, bodyweight):
    """
    Computes the Q points score for men

    total: The total hit in kg
    bodyweight: The body weight
    """
    bw_ratio = bodyweight / 100
    denominator = 416.70 - 47.87 * bw_ratio**(-2) + 18.93 * bw_ratio**2
    return (total * 463.26) / denominator

def q_points_female(total, bodyweight):
    """
    Computes the Q points score for woman

    total: The total hit in kg
    bodyweight: The body weight
    """
    bw_ratio = bodyweight / 100
    denominator = 266.50 - 19.44 * bw_ratio**(-2) + 18.61 * bw_ratio**2
    return (total * 306.54) / denominator

def qbw_male(bodyweight):
    """
    Computes the Q point bodyweight factor for men

    bodyweight: The body weight
    """
    bw_ratio = bodyweight / 100
    denominator = 416.70 - 47.87 * bw_ratio**(-2) + 18.93 * bw_ratio**2
    return 463.26 / denominator

def qbw_female(bodyweight):
    """
    Computes the Q points bodyweight factor for woman

    bodyweight: The body weight
    """
    bw_ratio = bodyweight / 100
    denominator = 266.50 - 19.44 * bw_ratio**(-2) + 18.61 * bw_ratio**2
    return 306.54 / denominator

def sn_cj_determiner(total, ratio):
    """
    Determines the snatch and clean and jerk from the ratio given of snatch/clean and jerk
    Returns both snatch and clean and jerk to the nearest integer

    total: The total we are dictating for
    ratio: Is the ration of snatch/clean and jerk
    """
    Rat_mat = np.array([[1, 1], [1, -ratio]])
    tot_vec = np.array([total, 0])
    sn_cj = np.linalg.solve(Rat_mat, tot_vec)
    return int(sn_cj[0]), int(sn_cj[1]) #Snatch is first, then clean and jerk

def sn_cj_compute(totals, ratio, brac):
    """
    Builds a list of snatches, clean and jerks, and totals from the given standards to be appended into the dataframe
    returns a list of the form [sn1, cj1, tot1, sn2, cj2, tot2, sn3, ....]

    totals: The list of standards as an array
    ratio: The ratio of snatch/clean and jerk
    brac: A boolian statement that dictates wether we want each one in brackets. True will give in brackets while false will not
    """
    records = []
    for i in range(len(totals)):
        snatch, clandjk = sn_cj_determiner(totals[i], ratio)
        if brac:
            records = records + [[snatch], [clandjk], [totals[i]]]
        else:
            records = records + [snatch, clandjk, totals[i]]
    return records

def rep_lift_colbuild(totals, lifts):
    """
    Builds a list of each lift to match the records

    totals: The list of totals we need the lifts listed for in the record sheets
    lifts: A lift of strings that we the records for. This includes the total entry. Make sure this is in order for how you want it in the record sheet
    """
    lift_list = []
    for i in range(len(totals)):
        for j in range(len(lifts)):
            lift_list.append(lifts[j])
    return lift_list

def standard_percentage(totals, scale_perc):
    """
    Scalining the world standards baised off of our scale factor

    totals: The list of totals we want to scale
    scale_perc: the percentage we want to scale by
    """
    sc_stnds = np.array(totals)*scale_perc*.01
    sc_stnds_ints = []
    for i in range(len(sc_stnds)):
        sc_stnds_ints.append(int(sc_stnds[i]))
    return sc_stnds_ints

def rep_col_build(string, count):
    """
    Builds a array of the same repeating element. This is usefull for columns that have all of the same value

    string: The element we want to repeat
    count: how many times we want to repeat
    """
    rep_list = []
    for i in range(count):
        rep_list.append(string)
    return rep_list

def wt_class_rep(wt_clss, lifts):

    """
    Builds the weightclass column in the dataframe.

    wt_clss: the list of weightclasses
    lifts: The list of lifts including the total
    """
    wt_cls_rplist = []
    for i in range(len(wt_clss)):
        for j in range(len(lifts)):
            wt_cls_rplist.append(wt_clss[i])
    return wt_cls_rplist

def mst_agecat_build(Sex, Age):
    """
    Builds the masters age catagories

    Sex: The sex of the catagory as a string
    Age: The array of masters age catagories
    """
    age_cats = []
    for i in range(len(Age)):
        app_val = str(Sex) + str(Age[i])
        age_cats.append(app_val)
    return age_cats

def scale_factor_locator(scal_fact_df, age, sex):
    """
    Locates the masters scale factor we want to scale by


    scal_fact_df: The scale factor dataframe as defined above
    age: The intiger age we want to get the scale factor for
    sex: The sex of the athlete we want to get the scale factor for as M or F
    """
    scal_fact_df = scal_fact_df[scal_fact_df["Age"] == int(age)]
    scal_fact_df = scal_fact_df.reset_index(drop=True)
    if sex == "M":
        return scal_fact_df.at[0, "AgeFactor_Men"]
    elif sex == "W":
        return scal_fact_df.at[0, "AgeFactor_Women"]
    else:
        print("sex needs to be M or F, here comes an error")

def age_fac_wc_lift_builder(wt_clsss, mst_agecats, lifts):
    """
    Builds the columns for weightclasses, age catagories, and lifts for the masters dataframe

    wt_clsss: The array of masters weightclasses
    mst_age: The age classes for masters
    lifts: The array of snatch, clean and jerk, and total
    """
    wt_classes_out = []
    lifts_out = []
    mst_agecats_out = []
    for i in range(len(mst_agecats)):
        for j in range(len(wt_clsss)):
            for k in range(len(lifts)):
                lifts_out.append(lifts[k])
                wt_classes_out.append(wt_clsss[j])
                mst_agecats_out.append(mst_agecats[i])
    return wt_classes_out, lifts_out, mst_agecats_out

def kg_remover(wt_clsss):
    """
    Removes the "kg" part of the string from a list of weightclasses and returns the integer

    wt_clsss: The string of weightclasses
    """
    clean_wtclsses = []
    for i in range(len(wt_clsss)):
        clean_wtclss = int(wt_clsss[i].replace("kg", ""))
        clean_wtclsses.append(clean_wtclss)
    return clean_wtclsses

def kg_remover_str(wt_clsss):
    """
    Removes the "kg" part of the string from a list of weightclasses and return as a string

    wt_clsss: The string of weightclasses
    """
    clean_wtclsses = []
    for i in range(len(wt_clsss)):
        clean_wtclss = wt_clsss[i].replace("kg", "")
        clean_wtclsses.append(clean_wtclss)
    return clean_wtclsses

def str_remover(wt_clsss, str_i):
    """
    Removes a part of a string from a list of elements

    wt_clsss: The the list of strings
    str_i: The part of the string we are trying to remove as a string
    """
    clean_wtclsses = []
    for i in range(len(wt_clsss)):
        clean_wtclss = int(wt_clsss[i].replace(str_i, ""))
        clean_wtclsses.append(clean_wtclss)
    return clean_wtclsses

def rep_replace(wt_clsss, rem, repl):
    """
    Replaces am element in a string if it exists

    wt_clsss: The the list of strings
    rem: The part of the string we are trying to remove as a string
    repl: The part of the string we are trying to replace as a string
    """
    clean_wtclsses = []
    for i in range(len(wt_clsss)):
        clean_wtclss = wt_clsss[i].replace(str(rem), str(repl))
        clean_wtclsses.append(clean_wtclss)
    return clean_wtclsses

def q_points_tot_finder(totals, sex, ages, agefact_df, wt_classes):
    """
    Computes the scaled standards for the totals for each age and weightclasses

    totals: The totals we want to scale
    sex: The sex M or F of the record sheet
    ages: The masters ages
    agefact_df: The dataframe of scale factors factors
    wt_classes: The weightclasses
    """
    wt_classes_int = kg_remover(wt_classes)
    scaled_totals = []
    for i in range(len(ages)):
        age_fac_use = scale_factor_locator(agefact_df, ages[i], sex)
        for j in range(len(wt_classes_int)):
            if sex == "M":
                Q_score = q_points_male(totals[j], wt_classes_int[j])
                scaled_tot = Q_score / (qbw_male(wt_classes_int[j]) * age_fac_use)
                scaled_totals.append(int(scaled_tot))
            elif sex == "W":
                Q_score = q_points_female(totals[j], wt_classes_int[j])
                scaled_tot = Q_score / (qbw_female(wt_classes_int[j]) * age_fac_use)
                scaled_totals.append(int(scaled_tot))
    return scaled_totals

def maxage_compute(age_list, top, full):
    """
    Computes a list of maximum ages from a given set of minumum ages

    age_list: The list of minumum ages we want to compute the max age for. We assume this list is ordered and has at least 2 elements in it
    top: The very maximum age that we assume someone can be as an integer
    full: Indicates if we are dealing with an array that is of the full length of the record sheet. If so, this needs to be true
    """
    if full:
        age_list = sorted(list(set(age_list)))
    max_ages = []
    for i in range(1, len(age_list)):
        max_age = int(int(age_list[i]) - 1)
        max_ages.append(max_age)
    max_ages.append(top)
    return max_ages

def scale_up(the_list, wcs, lift):
    """
    Scales up the max age to fit the whole record sheet

    the_list: the list of max ages we want to scale up
    wcs: The amount of weightclasses
    lift: The lifts that we need to acount for including total
    """
    new_list = []
    for i in range(len(the_list)):
        for j in range(len(wcs)):
            for k in range(len(lift)):
                new_list.append(the_list[i])
    return new_list

def lowt_compute(the_list, full):
    """
    gets the array of low weights from the array of high weights

    the_list: The array of high weightclasses
    full: Specifies if it is the full list from the record sheet
    """
    if full:
        the_list = sorted(list(set(the_list)))
    if the_list:
        the_list.pop()
        the_list = sorted(set(the_list), key=lambda x: int(x)) #Resort any number ending with zero
    return [0] + the_list

def scale_upwts(wt_clsss, mst_agecats, lifts):
    """
    Scales up the list of lower weightclasses to fit the whole record sheet

    wt_clsss: the list of lower weightclasses we want to scale up
    mst_agecats: The age classes for masters
    lift: The lifts that we need to acount for including total
    """
    wt_classes_out = []
    for i in range(len(mst_agecats)):
        for j in range(len(wt_clsss)):
            for k in range(len(lifts)):
                wt_classes_out.append(wt_clsss[j])
    return wt_classes_out

#Define sheet building functions for AWA

def awa_ythjrsr_sheetbuild(totals, scale_perc, age_cat, lifts, wt_clss, sheet_colnames, sn_cjr):
    """
    Builds the jr, yth, or sr record sheet in AWA format

    totals: The list of totals we are using for the sheet unscaled. Input will be the canadian INT2 totals defined above
    scale_perc: The scale factor we want to scale by
    age_cat: The age catagory of the record sheet (yth, jr, sr)
    lifts: The lifts + total that the sheets is holding records for. It is a 3 item array in this case
    wt_class: The weghtclass's that we are keeping records for
    sheet_colnames: A list of column names for the record sheet in order
    sn_cjr: The ratio of snatch to clean and jerk as dictated above
    """
    #Start by building the needed columns
    scaled_stds = standard_percentage(totals, scale_perc)
    set_recs = sn_cj_compute(scaled_stds, sn_cjr, True)
    lifts_for_recs = rep_lift_colbuild(scaled_stds, lifts)
    age_cat_col = rep_col_build(age_cat, len(lifts_for_recs))
    wt_clss_col = wt_class_rep(wt_clss, lifts)

    #Build the dataframe
    rec_df = pd.DataFrame("", index = range(len(age_cat_col)), columns = sheet_colnames)
    rec_df["Category"] = age_cat_col
    rec_df["Lift"] = lifts_for_recs
    rec_df["Record"] = set_recs
    rec_df["Weight Cat."] = wt_clss_col

    return rec_df

#Build the AWA masters sheets

def awa_mst_sheetbuild(totals, scale_perc, mst_ages, lifts, wt_clss, sheet_colnames, sn_cjr, age_fact_df, sex):
    """
    Builds the masters record sheet in AWA format

    totals: Sr totals used for scaling
    scale_perc: The scale factor we want to scale by
    mst_ages: The age catagories used for masters athletes
    lifts: The lifts + total that the sheets is holding records for. It is a 3 item array in this case
    wt_class: The weghtclass's that we are keeping records for
    sheet_colnames: A list of column names for the record sheet in order
    sn_cjr: The ratio of snatch to clean and jerk as dictated above
    age_fact_df: The dataframe of age factors
    sex: The sex of the group as M or F
    """
    #Start by building the needed columns
    scaled_stds = standard_percentage(totals, scale_perc)
    mst_agecats_fin = mst_agecat_build(sex, mst_ages)
    mst_wtc, liftcol_out, extmstagecat_col = age_fac_wc_lift_builder(wt_clss, mst_agecats_fin, lifts)
    mod_totals = q_points_tot_finder(scaled_stds, sex, mst_ages, age_fact_df, wt_clss)
    set_recs = sn_cj_compute(mod_totals, sn_cjr, True)

    #Build the dataframe
    rec_df = pd.DataFrame("", index = range(len(set_recs)), columns = sheet_colnames)
    rec_df["Category"] = extmstagecat_col
    rec_df["Lift"] = liftcol_out
    rec_df["Record"] = set_recs
    rec_df["Weight Cat."] = mst_wtc

    return rec_df

#Define needed functions to import and modify excell sheets

#Function to auto-adjust column widths
def auto_adjust_columns(df, worksheet):
    for idx, col in enumerate(df.columns):
        #Determine the max width: max between column name and longest cell content
        max_len = max(
            df[col].astype(str).map(len).max(),  #Max length of data in column
            len(str(col))                        #Length of column name
            )
          #Set column width (add a little padding)
        worksheet.set_column(idx, idx, max_len + 2)

def add_horizontal_separators(df, worksheet):
    for row_idx in range(len(df)):
        excel_row = row_idx + 1  #Offset by 1 for the header row
        if row_idx != 0 and row_idx % 3 == 0:
            for col_idx in range(len(df.columns)):
                value = df.iat[row_idx, col_idx]
                worksheet.write(excel_row, col_idx, value, thick_top_border)


In [ ]:
#6)

#Build all of the sheets in AWA format and export if specified

#YTH
AWA_yth_m_sheet = awa_ythjrsr_sheetbuild(yth_M_st, yth_perc, "YTH", liftsAWA, yth_weight_classes_male, AB_format_cnames, sn_cj_rat)
#AWA_yth_m_sheet
AWA_yth_f_sheet = awa_ythjrsr_sheetbuild(yth_F_st, yth_perc, "YTH", liftsAWA, yth_weight_classes_female, AB_format_cnames, sn_cj_rat)
#AWA_yth_f_sheet

#JR
AWA_jr_m_sheet = awa_ythjrsr_sheetbuild(jr_M_st, jr_perc, "JR", liftsAWA, weight_classes_male, AB_format_cnames, sn_cj_rat)
#AWA_jr_m_sheet
AWA_jr_f_sheet = awa_ythjrsr_sheetbuild(jr_F_st, jr_perc, "JR", liftsAWA, weight_classes_female, AB_format_cnames, sn_cj_rat)
#AWA_jr_f_sheet

#SR
AWA_sr_m_sheet = awa_ythjrsr_sheetbuild(sr_M_st, sr_perc, "SR", liftsAWA, weight_classes_male, AB_format_cnames, sn_cj_rat)
#print(AWA_sr_m_sheet)
#AWA_sr_m_sheet
AWA_sr_f_sheet = awa_ythjrsr_sheetbuild(sr_F_st, sr_perc, "SR", liftsAWA, weight_classes_female, AB_format_cnames, sn_cj_rat)
#AWA_sr_f_sheet

#Build the AWA masters sheets

male_mst_stands = awa_mst_sheetbuild(sr_M_st, mast_perc, mst_ages, liftsAWA, weight_classes_male, AB_format_cnames, .8, agefac_dat, "M")
#print(male_mst_stands)

female_mst_stands = awa_mst_sheetbuild(sr_F_st, mast_perc, mst_ages, liftsAWA, weight_classes_female, AB_format_cnames, .8, agefac_dat, "W")
#print(female_mst_stands)


#Export the records sheets for AWA
if to_exp == True:

  #Export the yth, jr, sr records
  yth_jr_sr_AWA_name = "YthJrSR_records.xlsx"
  AWA_ythjrsr_path = AWA_new_rec_dump + "/" + yth_jr_sr_AWA_name
  with pd.ExcelWriter(AWA_ythjrsr_path, engine='xlsxwriter') as writer:
    AWA_yth_m_sheet.to_excel(writer, sheet_name='Men Yth', index=False)
    AWA_yth_f_sheet.to_excel(writer, sheet_name='Woman Yth', index=False)
    AWA_jr_m_sheet.to_excel(writer, sheet_name='Men Jr', index=False)
    AWA_jr_f_sheet.to_excel(writer, sheet_name='Women Jr', index=False)
    AWA_sr_m_sheet.to_excel(writer, sheet_name='Men Sr', index=False)
    AWA_sr_f_sheet.to_excel(writer, sheet_name='Woman Sr', index=False)

    workbook = writer.book
    thick_top_border = workbook.add_format({'top': 1})  #Border style
    border_format = workbook.add_format({'border': 1})  #Black border around all sides

    sheets = {'Men Yth': AWA_yth_m_sheet,
              'Woman Yth': AWA_yth_f_sheet,
              'Men Jr': AWA_jr_m_sheet,
              'Women Jr': AWA_jr_f_sheet,
              'Men Sr': AWA_sr_m_sheet,
              'Woman Sr': AWA_sr_f_sheet,}

    #Apply auto-adjustment to all sheets
    for sheet_name, df in sheets.items():
        worksheet = writer.sheets[sheet_name]
        auto_adjust_columns(df, worksheet)

  #Export the masters sheets
  mast_AWA_name = "MAST_records.xlsx"
  AWA_mast_path = AWA_new_rec_dump + "/" + mast_AWA_name
  with pd.ExcelWriter(AWA_mast_path, engine='xlsxwriter') as writer:
    male_mst_stands.to_excel(writer, sheet_name='Men Masters', index=False)
    female_mst_stands.to_excel(writer, sheet_name='Women Masters', index=False)

    workbook = writer.book
    men_worksheet = writer.sheets['Men Masters']
    women_worksheet = writer.sheets['Women Masters']

    auto_adjust_columns(male_mst_stands, men_worksheet)
    auto_adjust_columns(female_mst_stands, women_worksheet)
    